# **ИМПОРТ**

In [1]:
import itertools
from shutil import copyfile
import os
import datetime
import json
import pandas as pd
import numpy as np
import dill as pickle #более мощная библиотека позволяющая сохранять функции
import copy

from IPython.display import display
from matplotlib import pyplot as plt

import warnings
warnings.simplefilter("ignore")

%matplotlib inline
pd.set_option('display.max_columns', None)

%matplotlib inline

# **КЛАСС СБОРЩИКА АГРЕГАТОВ**

In [2]:
# Cборщик агрегатов
class AggCollector:
    
    def __init__ (self,
                  acct_type_list,    
                  period_dict,
                  PATH_TO_DATA,
                  account_csv_file_name,
                  information_csv_file_name,
                  name_csv_file_name,
                  address_csv_file_name,
                  cbrates_csv_file_name):
        
        
        self.acct_type_list = acct_type_list
        self.period_dict = period_dict
        
        # Путь и названия файлов с данными
        self.PATH_TO_DATA = PATH_TO_DATA
        
        self.account_csv_file_name = account_csv_file_name
        self.information_csv_file_name = information_csv_file_name
        self.name_csv_file_name = name_csv_file_name
        self.address_csv_file_name = address_csv_file_name
        self.cbrates_csv_file_name = cbrates_csv_file_name
        
        # Заголовки столбцов таблиц        
        self.account_cols = ['ACCT_RTE_CDE', 'ACCT_RTE_DTE', 'ACCT_TYPE', 'AMT_PAST_DUE', 'CLOSED_DT', 'REPORTING_DT',
                             'CREDIT_LIMIT', 'CURRENCY_CODE', 'FID', 'MEMBER_CODE', 'OPENED_DT', 'OWNER_INDIC',
                             'PAYMT_PAT', 'PAYMT_PAT_START_DT', 'PAYT_DUE_DTE', 'SERIAL_NUM']
        
        self.name_cols = ['FID', 'FILE_SINCE_DT', 'LAST_UPDATED_DT', 'MIDDLE', 'BIRTH_DT']
        self.info_cols = ['SERIAL_NUM', 'FID', 'APP_DATE', 'CREDITOR_TYPE', 'MEMBER_CODE', 'TYPE_FLAG', 'LOAN_TYPE',
                          'FLAG_OF_APPROVAL', 'REJECTED_AMT',  'REJ_AMT_CUR', 'REJECTION_DTE', 'REJECTION_REASON']
        self.addr_cols = ['FID', 'PROV', 'ADDR_REL_TYP_CDE', 'FILE_SINCE_DT', 'LAST_UPDATED_DT']       
        
        self.db_names = None
        self.db_cbrates = None
        self.db_addr = None
        self.account_keys = None
        self.info_keys = None
        
        self.df_keys = None
        self.df = None
        self.df_rejects = None
        self.db_account = None
        self.db_info = None
        
        self.df_test = None
        self.df_test_FID = None
        self.df_test_account = None
        self.df_test_info = None
        
        self.name = None
        self.log = ''
        

    # Функция логирования
    def print_log(self, log_str, level=0, timestamp=True):
        
        if not self.name:
            self.name = [name for name in globals() if globals()[name] == self][0]
        
        now_str = lambda : datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if timestamp:
            log_str = '{:.<70}   {}\n'.format(level * 4 * ' ' + log_str + '   ', now_str()) 
        else:
            log_str = level * 4 * ' ' + log_str + '\n'

        # Запись в лог экземпляра класса 
        self.log += log_str
        
        # Запись в лог файл
        with open(self.PATH_TO_DATA + self.name + '.log', 'a', encoding='utf-8') as f:
            f.write(log_str)

        # Вывод сообещения на экран
        print(log_str, end='')
        
        
    # Формирование файлов с агрегатами из сырх данных
    def collect(self, periods_to_collect_list='all', check_data_export=False, collect_rejects=False):
        self.print_log('РАСЧЕТ АГРЕГАТОВ\n', level=0, timestamp=False)
        self.load_account_keys()
        self.laod_info_keys()
        self.process_db_addr()
        self.process_db_names()
        self.process_db_cbrates()
        
        if periods_to_collect_list=='all':
            periods_to_collect_list = list(self.period_dict.keys())
        
        for period in periods_to_collect_list:
            
            start_date, finish_date = self.period_dict[period][0], self.period_dict[period][1]
            self.print_log('\nПЕРИОД {} - {}:'.format(start_date, finish_date), level=0, timestamp=False)
            
            # Предобработка данных
            self.get_df_keys(start_date=start_date, finish_date=finish_date, portion=1)
            self.process_db_account(start_date, finish_date)
            self.process_db_info()
            
            # Формирование датасета для сбора агрегатов по отказникам (обрезание датасета до размера )
            if collect_rejects:
                self.keep_rejects_only()
            
            # Формирование списка тестовых FID
            if self.df.shape[0] < 2000:
                chek_data_qty = self.df.shape[0]
            else: 
                chek_data_qty = 2000
            
            self.df_test_FID = self.df.sample(chek_data_qty, replace=False, random_state=42)['FID'].unique()
            
            # Сбор агрегатов и расчет флагов
            self.agg_db_address()
            self.agg_db_names()
            self.agg_db_info()            
            self.agg_db_account()
            self.flag_calculation()
            
            # Экспорт файла с агрегатами            
            if collect_rejects:
                df_file_name = 'REJECT_AGG_' + period + '.csv'
                # Добавление информации об отказах
                self.df = pd.merge(self.df,
                                   self.df_rejects,
                                   how='inner',
                                   on=['SERIAL_NUM'],
                                   suffixes= ('', '_REJECT'))
            else:
                df_file_name = 'AGG_' + period + '.csv'
            
            self.print_log('Выгрузка в файл ' + df_file_name, level=1, timestamp=True)  
            self.df.to_csv(self.PATH_TO_DATA + df_file_name, index=False)
            self.print_log('Выгрузка в файл завершена', level=1, timestamp=True)
            
            # Экспорт проверочных данных
            if check_data_export:
                if collect_rejects:
                    check_file_name = 'REJECT_AGG_' + period + '_checkdata.xlsx'
                else:
                    check_file_name = 'AGG_' + period + '_checkdata.xlsx'
                
                self.print_log('Сохранение файла ' + check_file_name, level=1, timestamp=True) 
                
                writer = pd.ExcelWriter(self.PATH_TO_DATA + check_file_name)
                
                self.df_test = self.df[self.df['FID'].isin(self.df_test_FID)]
                self.df_test.to_excel(writer, sheet_name='AGG', engine='io.excel.xlsx.writer')                
                self.df_test_account.to_excel(writer, sheet_name='ACCOUNT', engine='io.excel.xlsx.writer')
                self.df_test_info.to_excel(writer, sheet_name='INFORMATION', engine='io.excel.xlsx.writer')
                writer.save()
                
                self.print_log('Проверочный файл сохранен', level=1, timestamp=True)
            
            # Удаление всех временных переменных
            self.df_keys = None
            self.df = None
            self.df_rejects = None
            self.db_account = None
            self.db_info = None
            self.df_test = None
            self.df_test_FID = None
            self.df_test_account = None
            self.df_test_info = None
    

    # Формирование тестового набора данных
    def get_sample_data(self, portion=0.05):
        
        self.print_log('ПОДГОТОВКА СЭМПЛА ДАННЫХ\n', level=0, timestamp=False)        
        
        self.load_account_keys()
        self.laod_info_keys()
        
        #Формирование списка сэмпловых FID
        periods_to_collect_list = list(self.period_dict.keys())
        
        self.print_log('\nСписок целевых FID', level=0, timestamp=False)
        
        for period in periods_to_collect_list:
            start_date, finish_date = self.period_dict[period][0], self.period_dict[period][1]       

            
            self.get_df_keys(start_date=start_date, finish_date=finish_date, portion=portion)
            
            if period == periods_to_collect_list[0]:
                df_keys = self.df_keys.copy()
            else:
                df_keys = pd.concat((df_keys, self.df_keys),axis=0)
        
        sample_FID = df_keys['FID'].unique()
        del(df_keys)
        self.df_keys = None
        
        # Функция для формирования полного пути файла семпла
        sample_file_path = lambda s: self.PATH_TO_DATA + 'SAMPLE/' + s[:-4] + '_SAMPLE' + '.csv'
        
        # Создание папки для сохранения файлов
        if not os.path.exists(self.PATH_TO_DATA + 'SAMPLE/'):
            os.makedirs(self.PATH_TO_DATA + 'SAMPLE/')
        
        # Копирование таблицы ACCOUNT            
        self.print_log('ACCOUNT Выборка ключей для загрузки', level=0, timestamp=True)
        account_skip_rows = self.account_keys[~self.account_keys['FID'].isin(sample_FID)].index

        self.print_log('ACCOUNT Загрузка данных', level=0, timestamp=True)
        db_account = pd.read_csv(self.PATH_TO_DATA + self.account_csv_file_name,
                                 encoding='windows-1251',
                                 sep=';',
                                 quotechar = '"',
                                 names=self.account_cols,
                                 skiprows=account_skip_rows)
        
        self.print_log('ACCOUNT Cохранение сэмпла данных', level=0, timestamp=True)        
        db_account.to_csv(sample_file_path(self.account_csv_file_name),
                          encoding='windows-1251',
                          sep=';',
                          quotechar = '"',
                          header=False,
                          index=False)
        del(db_account)
        
        
        # Копирование таблицы INFORMATION                
        self.print_log('INFORMATION Выборка ключей для загрузки', level=0, timestamp=True)
        info_skip_rows = self.info_keys[~self.info_keys['FID'].isin(sample_FID)].index

        self.print_log('INFORMATION Загрузка данных', level=0, timestamp=True)
        db_info = pd.read_csv(self.PATH_TO_DATA + self.information_csv_file_name,
                              encoding='windows-1251',
                              sep=';',
                              quotechar = '"',
                              names=self.info_cols,
                              skiprows=info_skip_rows)
        
        self.print_log('INFORMATION Cохранение сэмпла данных', level=0, timestamp=True)        
        db_info.to_csv(sample_file_path(self.information_csv_file_name),
                       encoding='windows-1251',
                       sep=';',
                       quotechar = '"',
                       header=False,
                       index=False)
        del(db_info)
        
        
        # Копирование таблицы ADDRESS        
        self.print_log('ADDRESS Загрузка данных', level=0, timestamp=True)
        db_addr = pd.read_csv(self.PATH_TO_DATA + self.address_csv_file_name,
                              encoding='windows-1251',
                              sep=';',
                              quotechar = '"',
                              names=self.addr_cols)            
        
        self.print_log('ADDRESS Выборка данных', level=0, timestamp=True)
        db_addr = db_addr[db_addr['FID'].isin(sample_FID)]
        
        self.print_log('ADDRESS Cохранение сэмпла данных', level=0, timestamp=True)
        db_addr.to_csv(sample_file_path(self.address_csv_file_name),
                       encoding='windows-1251',
                       sep=';',
                       quotechar = '"',
                       header=False,
                       index=False)        
        del(db_addr)
        
        
        # Копирование таблицы NAME
        self.print_log('NAME Загрузка таблицы', level=0, timestamp=True)
        db_names = pd.read_csv(self.PATH_TO_DATA + self.name_csv_file_name,
                               names=self.name_cols,
                               encoding='windows-1251',
                               sep=';',
                               quotechar = '"')
        
        self.print_log('NAME Выборка данных', level=0, timestamp=True)
        db_names = db_names[db_names['FID'].isin(sample_FID)]
        
        self.print_log('NAME Cохранение сэмпла данных', level=0, timestamp=True)           
        db_names.to_csv(sample_file_path(self.name_csv_file_name),
                        encoding='windows-1251',
                        sep=';',
                        quotechar = '"',
                        header=False,
                        index=False)
        del(db_names)
        
        
        # Копирование таблицы CBRATES
        copyfile(self.PATH_TO_DATA + self.cbrates_csv_file_name, sample_file_path(self.cbrates_csv_file_name))
        
        get_sample_data_result = (self.acct_type_list,
                                  self.period_dict,
                                  self.PATH_TO_DATA + 'SAMPLE/',
                                  self.account_csv_file_name[:-4] + '_SAMPLE' + '.csv',
                                  self.information_csv_file_name[:-4] + '_SAMPLE' + '.csv',
                                  self.name_csv_file_name[:-4] + '_SAMPLE' + '.csv',
                                  self.address_csv_file_name[:-4] + '_SAMPLE' + '.csv',
                                  self.cbrates_csv_file_name[:-4] + '_SAMPLE' + '.csv')
        
        return get_sample_data_result
    
    # Формирование списка целевых FID
    def get_df_keys(self, start_date, finish_date, portion=1):
        
        self.print_log('ACCOUNT Сбор FID (' + start_date + ' - ' + finish_date + ')', level=1, timestamp=True)
        
        start_date_int = int(start_date[:4] + start_date[5:7] + start_date[8:])
        finish_date_int = int(finish_date[:4] + finish_date[5:7] + finish_date[8:])
        
        self.df_keys = self.account_keys[(self.account_keys['OPENED_DT'] >= start_date_int) &
                                         (self.account_keys['OPENED_DT'] <= finish_date_int) &
                                         (self.account_keys['OWNER_INDIC'].isin([1, 4])) &
                                         (self.account_keys['ACCT_TYPE'].isin(self.acct_type_list))]        
        
        if portion < 1:
            self.df_keys = self.df_keys.sample(int(self.df_keys.shape[0] * portion), replace=False, random_state=42)

    
    # Загрузка ключей Account    
    def load_account_keys(self):
        if str(type(self.account_keys)) == "<class 'NoneType'>":
            self.print_log('ACCOUNT Загрузка ключей таблицы', level=0, timestamp=True)
            self.account_keys = pd.read_csv(self.PATH_TO_DATA + self.account_csv_file_name,
                                            encoding='windows-1251',
                                            sep=';',
                                            quotechar = '"',
                                            names=self.account_cols,
                                            usecols=['FID', 'OPENED_DT', 'OWNER_INDIC', 'ACCT_TYPE'])
    
    
    # Загрузка ключей Information    
    def laod_info_keys(self):
        # Загрузка ключей Information        
        if str(type(self.info_keys)) == "<class 'NoneType'>":        
            self.print_log('INFORMATION Загрузка ключей таблицы', level=0, timestamp=True)
            self.info_keys = pd.read_csv(self.PATH_TO_DATA + self.information_csv_file_name,
                                         encoding='windows-1251',
                                         sep=';',
                                         quotechar = '"',
                                         names=self.info_cols,
                                         usecols=['FID'])
     
    
    # Загрузка и обработка Name   
    def process_db_names (self):

        if str(type(self.db_names)) == "<class 'NoneType'>":
            self.print_log('NAME Загрузка таблицы', level=0, timestamp=True)
            db_names = pd.read_csv(self.PATH_TO_DATA + self.name_csv_file_name,
                                   names=self.name_cols,
                                   encoding='windows-1251',
                                   sep=';',
                                   quotechar = '"',
                                   converters={'LAST_UPDATED_DT':lambda s: s[:10]})


            self.print_log('NAME Предобработка таблицы', level=0, timestamp=True)
            db_names['LAST_UPDATED_DT'] = pd.to_datetime(db_names['LAST_UPDATED_DT'], errors='coerce')
            db_names['FILE_SINCE_DT'] = pd.to_datetime(db_names['FILE_SINCE_DT'], format='%Y%m%d', errors='coerce')        
            db_names['BIRTH_DT'] = pd.to_datetime(db_names['BIRTH_DT'], format='%Y%m%d', errors='coerce')

            # Удаление данных где неадекватные даты
            db_names = db_names[~db_names['LAST_UPDATED_DT'].isnull()]
            db_names = db_names[~db_names['FILE_SINCE_DT'].isnull()]        
            db_names = db_names[~db_names['BIRTH_DT'].isnull()]

            get_gender = lambda x: 1 if (str(x)[-2:] in ['НА','ЗЫ', 'на', 'зы']) else (0 if (str(x)[-2:] in ['ИЧ','ЛЫ', 'ич', 'лы']) else None)
            db_names['GENDER'] = db_names['MIDDLE'].map(get_gender)    
            
            self.db_names = db_names

    
    # Загрузка и обработка CBRates 
    def process_db_cbrates (self):
        if str(type(self.db_cbrates)) == "<class 'NoneType'>":
            self.print_log('CBRATES Загрузка таблицы', level=0, timestamp=True)
            db_cbrates = pd.read_csv(self.PATH_TO_DATA + self.cbrates_csv_file_name,
                                     sep=';',
                                     quotechar = "'",
                                     decimal=",",
                                     index_col='ID')

            self.print_log('CBRATES Предобработка таблицы', level=0, timestamp=True)

            # Удаление лишних валют из таблицы валют
            #db_cbrates = db_cbrates.loc[db_cbrates['CODE'].isin(db_account['CURRENCY_CODE'].value_counts().index)]

            # Расчет реального курса валюты (с учетом номинала)
            db_cbrates['CURRENCY_RATE'] = db_cbrates['RATE'] / db_cbrates['NOMINAL']
            db_cbrates['CURRENCY_CODE'] = db_cbrates['CODE'] 

            db_cbrates.drop(columns=['NOMINAL', 'RATE', 'CODE'], inplace=True)

            # Добавление месяца и года курса
            db_cbrates['DATE'] = pd.to_datetime(db_cbrates['DATE'], dayfirst=True)
            db_cbrates['MONTH'] = db_cbrates['DATE'].dt.month
            db_cbrates['YEAR'] = db_cbrates['DATE'].dt.year

            # Групировка курсов по месяцам
            db_cbrates = db_cbrates.groupby(by=['CURRENCY_CODE', 'YEAR', 'MONTH']).agg({'CURRENCY_RATE':np.mean})
            db_cbrates = db_cbrates.reset_index().sort_values(by=['YEAR', 'MONTH', 'CURRENCY_CODE'])

            # Добавление строк с курсами на все года с 1996-2005 (добавляется самый ранний известнрый курс)
            for year in range(1996,2005):
                for month in range(1,13):
                    for curr in db_cbrates['CURRENCY_CODE'].unique():
                        db_cbrates.loc[db_cbrates.shape[0]] = [curr, year, month, db_cbrates.loc[db_cbrates['CURRENCY_CODE']==curr,'CURRENCY_RATE'].iloc[0]]

            # Добавление курса рубля к рублю
            for year in range(1996,2025):
                for month in range(1,13):
                    db_cbrates.loc[db_cbrates.shape[0]] = ['RUB', year, month, 1]

            db_cbrates = db_cbrates[['YEAR', 'MONTH', 'CURRENCY_CODE', 'CURRENCY_RATE']].sort_values(by=['YEAR', 'MONTH', 'CURRENCY_CODE'])
            db_cbrates.index = np.arange(db_cbrates.shape[0])

            self.db_cbrates = db_cbrates
    
    
    # Загрузка и обработка Address 
    def process_db_addr (self):
        
        if str(type(self.db_addr)) == "<class 'NoneType'>":
        
            self.print_log('ADDRESS Загрузка таблицы', level=0, timestamp=True)
            db_addr = pd.read_csv(self.PATH_TO_DATA + self.address_csv_file_name,
                                  encoding='windows-1251',
                                  sep=';',
                                  quotechar = '"',
                                  names=self.addr_cols)

            self.print_log('ADDRESS Предобработка данных', level=0, timestamp=True)

            # Удаление типов несоответствующих 1 и 2
            db_addr = db_addr.loc[db_addr['ADDR_REL_TYP_CDE'].isin([1, 2, '1', '2'])]

            # Удаление пустых значений
            db_addr = db_addr[~(db_addr['PROV']=='  ')]
            db_addr = db_addr[~db_addr['PROV'].isnull()]

            # Преобразование типов
            db_addr['LAST_UPDATED_DT'] = pd.to_datetime(db_addr['LAST_UPDATED_DT'].map(lambda s:s[:10]), errors='coerce')
            db_addr['FILE_SINCE_DT'] = pd.to_datetime(db_addr['FILE_SINCE_DT'], format='%Y%m%d', errors='coerce')
            db_addr['ADDR_REL_TYP_CDE'] = db_addr['ADDR_REL_TYP_CDE'].astype('int32')

            # Удаление данных где неадекватные даты
            db_addr = db_addr[~db_addr['LAST_UPDATED_DT'].isnull()]
            db_addr = db_addr[~db_addr['FILE_SINCE_DT'].isnull()]            

            # Отсечение дуюлирующих строк
            db_addr = db_addr.groupby(by=['FID', 'PROV','ADDR_REL_TYP_CDE']).agg({'FILE_SINCE_DT':min, 'LAST_UPDATED_DT':max}).reset_index()

            self.db_addr = db_addr
   

    # Загрузка и предобработа данных таблицы ACCOUNT
    def process_db_account (self, start_date, finish_date):

        self.print_log('ACCOUNT Выборка ключей для загрузки', level=1, timestamp=True)
        account_skip_rows = self.account_keys[~self.account_keys['FID'].isin(self.df_keys['FID'])].index

        self.print_log('ACCOUNT Загрузка данных', level=1, timestamp=True)
        db_account = pd.read_csv(self.PATH_TO_DATA + self.account_csv_file_name,
                                 encoding='windows-1251',
                                 sep=';',
                                 quotechar = '"',
                                 names=self.account_cols,
                                 skiprows=account_skip_rows)        

        self.print_log('ACCOUNT Предобработка данных', level=1, timestamp=True)
        # Удаление записей в которых пустые значение ACCT_TYPE, PAYMT_PAT,CREDIT_LIMIT, ACCT_RTE_CDE, OPENED_DT
        db_account = db_account.loc[~db_account['ACCT_TYPE'].isnull()]
        db_account = db_account.loc[~db_account['PAYMT_PAT'].isnull()]
        db_account = db_account.loc[~db_account['CREDIT_LIMIT'].isnull()]
        db_account = db_account.loc[~db_account['ACCT_RTE_CDE'].isnull()]

        # Удаление FID по OWNER_INDIC не равен 1 или 4 
        db_account = db_account[db_account['OWNER_INDIC'].isin([1, 4])]

        # Заполнение пропусков
        db_account['AMT_PAST_DUE'].fillna(0, inplace=True)

        # Преобразование типов
        db_account['FID'] = db_account['FID'].astype('int')
        db_account['OWNER_INDIC'] = db_account['OWNER_INDIC'].astype('int')
        db_account['ACCT_RTE_CDE'] = db_account['ACCT_RTE_CDE'].astype('int')
        db_account['SERIAL_NUM'] = db_account['SERIAL_NUM'].astype('int')
        db_account['CREDIT_LIMIT'] =db_account['CREDIT_LIMIT'].astype('int64')
        db_account['AMT_PAST_DUE'] = db_account['AMT_PAST_DUE'].astype('int64')
        db_account['ACCT_TYPE'] = db_account['ACCT_TYPE'].astype('int')

        # удаление днных по нерелевантным тимпам кредитов 
        db_account = db_account[db_account['ACCT_TYPE'].isin([1,4,6,7,9,13,16,17,18])]

        # Удаление FID которых нет в таблице db_names
        db_account = db_account.loc[db_account['FID'].isin(self.db_names['FID'])]

        #Удаление FID по которым колв-о кредитов больше 100 
        num=100
        db_account = db_account[~db_account['FID'].isin(db_account['FID'].value_counts()[db_account['FID'].value_counts() > num].index)]

        #Даты
        db_account['ACCT_RTE_DTE'] = pd.to_datetime(db_account['ACCT_RTE_DTE'], format='%Y%m%d', errors='coerce')
        
        db_account['CLOSED_DT'] = pd.to_datetime(db_account['CLOSED_DT'], format='%Y%m%d', errors='coerce')
        db_account['OPENED_DT'] = pd.to_datetime(db_account['OPENED_DT'], format='%Y%m%d', errors='coerce')
        db_account['REPORTING_DT'] = pd.to_datetime(db_account['REPORTING_DT'], format='%Y%m%d', errors='coerce')
        db_account['PAYMT_PAT_START_DT'] = pd.to_datetime(db_account['PAYMT_PAT_START_DT'], format='%Y%m%d', errors='coerce')
        db_account['PAYT_DUE_DTE'] = pd.to_datetime(db_account['PAYT_DUE_DTE'], format='%Y%m%d', errors='coerce')

        # Удаление данных где неадекватные даты
        db_account = db_account[~db_account['ACCT_RTE_DTE'].isnull()]
        db_account = db_account[~db_account['OPENED_DT'].isnull()]
        db_account = db_account[~db_account['PAYMT_PAT_START_DT'].isnull()]
        #db_account = db_account[~db_account['PAYT_DUE_DTE'].isnull()]
        #db_account = db_account[~db_account['REPORTING_DT'].isnull()]        
        
        # Добавление месяца и года открытия кредита
        db_account['OPENED_DT_WEEKDAY'] = db_account['OPENED_DT'].dt.weekday
        db_account['OPENED_DT_DAY'] = db_account['OPENED_DT'].dt.day
        db_account['OPENED_DT_MONTH'] = db_account['OPENED_DT'].dt.month
        db_account['OPENED_DT_YEAR'] = db_account['OPENED_DT'].dt.year


        # преобразование MEMBER_CODE
        db_account['MEMBER_CODE'] = db_account['MEMBER_CODE'].map(lambda x: x[:6])    

        #Добавление курса валют в db_account и расчет CREDIT_LIMIT_RUB
        db_account = pd.merge(db_account, 
                              self.db_cbrates, 
                              how='left', 
                              left_on=['CURRENCY_CODE', 'OPENED_DT_YEAR', 'OPENED_DT_MONTH'], 
                              right_on=['CURRENCY_CODE', 'YEAR', 'MONTH'])

        db_account.drop(columns=['YEAR', 'MONTH', 'OPENED_DT_YEAR'], inplace=True)
        db_account['CREDIT_LIMIT_RUB'] = db_account['CREDIT_LIMIT'] * db_account['CURRENCY_RATE']
        db_account['AMT_PAST_DUE_RUB'] = db_account['AMT_PAST_DUE'] * db_account['CURRENCY_RATE'] 
        
        self.db_account = db_account

        self.print_log('Формирование датасета для сбора агргатов', level=1, timestamp=True)
        self.df = self.db_account[(self.db_account['OPENED_DT']>=start_date) &
                                  (self.db_account['OPENED_DT']<=finish_date) &
                                  (self.db_account['ACCT_TYPE'].isin(self.acct_type_list))].copy()
        
        
    # Предобработка данных таблицы INFORMATION
    def process_db_info (self):

        self.print_log('INFORMATION Выборка ключей для загрузки', level=1, timestamp=True)
        info_skip_rows = self.info_keys[~self.info_keys['FID'].isin(self.df_keys['FID'])].index

        self.print_log('INFORMATION Загрузка данных', level=1, timestamp=True)          
        db_info = pd.read_csv(PATH_TO_DATA + self.information_csv_file_name,
                              encoding='windows-1251',
                              sep=';',
                              quotechar = '"',
                              names=self.info_cols,
                              skiprows=info_skip_rows)

        self.print_log('INFORMATION Предобработка данных', level=1, timestamp=True)    
        db_info['REJECTED_AMT'].fillna(0, inplace=True)

        # Удаление строк где не указан тип кредита
        db_info = db_info.loc[~db_info['LOAN_TYPE'].isnull()]

        # Удаление FID которых нет TYPE_FLAG <> 1
        db_info = db_info[db_info['TYPE_FLAG']==1]

        # Преобразование типов
        db_info['APP_DATE'] = pd.to_datetime(db_info['APP_DATE'], format='%Y%m%d', errors='coerce')
        db_info['REJECTION_DTE'] = pd.to_datetime(db_info['REJECTION_DTE'], format='%Y%m%d', errors='coerce')
        
        db_info['REJECTED_AMT'] = db_info['REJECTED_AMT'].astype('int64')
        db_info['CREDITOR_TYPE'] = db_info['CREDITOR_TYPE'].astype('int32')
        db_info['TYPE_FLAG'] = db_info['TYPE_FLAG'].astype('int32')
        db_info['LOAN_TYPE'] = db_info['LOAN_TYPE'].astype('int32')
        
        # преобразование MEMBER_CODE
        db_info['MEMBER_CODE'] = db_info['MEMBER_CODE'].map(lambda x: x[:6])    

        # Удаление данных где неадекватные даты
        db_info = db_info[~db_info['APP_DATE'].isnull()]    

        # Замена пустых значений валюты на RUB (REJECTED_AMT - в этих случаях равен 0) 
        db_info['REJ_AMT_CUR'].loc[db_info['REJ_AMT_CUR'].isin(['   ', 'RU ', 'РУБ', 'RUR'])] = 'RUB'

        #Перерасчет суммы отказа в рубли в таблице db_info
        db_info['YEAR'] = db_info['APP_DATE'].dt.year
        db_info['MONTH'] = db_info['APP_DATE'].dt.month

        db_info = pd.merge(db_info, 
                           self.db_cbrates, 
                           how='left', 
                           left_on=['REJ_AMT_CUR', 'YEAR', 'MONTH'], 
                           right_on=['CURRENCY_CODE', 'YEAR', 'MONTH'])

        db_info['REJECTED_AMT_RUB'] = (db_info['REJECTED_AMT'] * db_info['CURRENCY_RATE'])
        db_info.drop(columns=['REJECTED_AMT', 'CURRENCY_CODE','CURRENCY_RATE', 'YEAR', 'MONTH'], inplace=True)

        self.db_info = db_info
    
    
    # Формирование датасета для сбора агрегатов по отказникам
    def keep_rejects_only (self):
        
        self.print_log('Фильтрация данных по отказникам', level=1, timestamp=True)
        
        db_info_rej = self.db_info[self.db_info['FLAG_OF_APPROVAL'] != 'Y']
        db_info_rej = db_info_rej[db_info_rej['REJECTED_AMT_RUB']>0]
        db_info_rej = db_info_rej[~db_info_rej['REJECTION_DTE'].isnull()]
        db_info_rej['REJECT_MEMBER_CODE'] = db_info_rej['MEMBER_CODE']
        db_info_rej['REJECT_LOAN_TYPE'] = db_info_rej['LOAN_TYPE']
        db_info_rej['REJECT_DTE'] = db_info_rej['REJECTION_DTE']
        db_info_rej['REJECT_REASON'] = db_info_rej['REJECTION_REASON']
        db_info_rej['REJECT_AMT_RUB'] = db_info_rej['REJECTED_AMT_RUB']
                
        db_info_rej = db_info_rej[['FID', 'REJECT_MEMBER_CODE', 'REJECT_DTE', 'REJECT_LOAN_TYPE', 'REJECT_REASON', 'REJECT_AMT_RUB']]
        
        df_tmp = pd.merge(self.df[['FID', 'SERIAL_NUM', 'OPENED_DT', 'ACCT_TYPE']],
                          db_info_rej,
                          how='inner',
                          on=['FID'],
                          suffixes= ('', '_REJECT'))
        
        del(db_info_rej)
        
        df_tmp['REJECT_DAYS_GAP'] = (df_tmp['OPENED_DT'] - df_tmp['REJECT_DTE']).dt.days 
        df_tmp = df_tmp[df_tmp['REJECT_DAYS_GAP'] >= 0]
        df_tmp = df_tmp[df_tmp['REJECT_DAYS_GAP'] <= 90]
        
        loan_type_30d = [201, 202, 203, 204, 301, 302, 303, 304, 305, 306, 401, 402, 403, 404, 405, 406, 407, 408]
        df_tmp = df_tmp[((df_tmp['REJECT_DAYS_GAP'] <= 90) & (df_tmp['REJECT_LOAN_TYPE'].isin([101, 102])) & (df_tmp['ACCT_TYPE'] == 1)) |
                        ((df_tmp['REJECT_DAYS_GAP'] <= 90) & (df_tmp['REJECT_LOAN_TYPE'].isin([501])) & (df_tmp['ACCT_TYPE'] == 6)) | 
                        ((df_tmp['REJECT_DAYS_GAP'] <= 30) & (df_tmp['REJECT_LOAN_TYPE'].isin(loan_type_30d)) & (df_tmp['ACCT_TYPE'].isin([16, 7, 9])))]
        
        df_tmp.drop_duplicates(inplace=True)

        self.df = self.df[self.df['SERIAL_NUM'].isin(df_tmp['SERIAL_NUM'])]
        self.df_rejects = df_tmp[['SERIAL_NUM', 'REJECT_MEMBER_CODE', 'REJECT_DTE', 'REJECT_LOAN_TYPE', 'REJECT_REASON', 'REJECT_AMT_RUB']].copy()
        del(df_tmp)
    
    
    # Функция добавление агрегатов REGION_REG и REGION_PR
    def agg_db_address(self):
        
        self.print_log('ADDRESS Сбор агрегатов', level=1, timestamp=True)
        df_tmp = pd.merge(self.df[['SERIAL_NUM', 'FID', 'OPENED_DT']],
                          self.db_addr,
                          how='left',
                          on=['FID'],
                          suffixes= ('', '_ADR'))
        df_tmp = df_tmp.loc[df_tmp['OPENED_DT'] >= df_tmp['FILE_SINCE_DT']]
        df_tmp.sort_values(by=['SERIAL_NUM', 'FILE_SINCE_DT'], ascending=[True, False], inplace=True)
        df_tmp.drop_duplicates(subset=['SERIAL_NUM', 'ADDR_REL_TYP_CDE'], keep='first', inplace=True)

        df_tmp['REGION_REG'] = 0
        df_tmp['REGION_PR'] = 0
        df_tmp['REGION_REG'].loc[df_tmp['ADDR_REL_TYP_CDE']==1] = df_tmp['PROV']
        df_tmp['REGION_PR'].loc[df_tmp['ADDR_REL_TYP_CDE']==2] = df_tmp['PROV']

        df_tmp['REGION_REG']  = df_tmp['REGION_REG'].astype('int')
        df_tmp['REGION_PR']  = df_tmp['REGION_PR'].astype('int')

        df_tmp = df_tmp.groupby(by=['SERIAL_NUM']).agg({'REGION_REG':np.sum, 'REGION_PR':np.sum})

        self.df['REGION_PR'] = self.df['SERIAL_NUM'].map(df_tmp['REGION_PR'].loc[df_tmp['REGION_PR']!=0].to_dict())
        self.df['REGION_REG'] = self.df['SERIAL_NUM'].map(df_tmp['REGION_REG'].loc[df_tmp['REGION_REG']!=0].to_dict())
        
        del(df_tmp)
    
    
    # Функция добавление агрегатов FID_DATE и FID_DAYS
    def agg_db_names(self):
        
        self.print_log('NAME Сбор агрегатов', level=1, timestamp=True)
        
        self.db_names.sort_values(by=['FID', 'FILE_SINCE_DT'], ascending=[True,True], inplace=True)
        df_tmp = self.db_names.drop_duplicates(subset=['FID'], keep='first')
        df_tmp = df_tmp[df_tmp['FID'].isin(self.df['FID'])]

        self.df['FID_DATE'] = self.df['FID'].map(df_tmp.set_index('FID')['FILE_SINCE_DT'].to_dict())
        self.df['FID_DAYS'] = (self.df['OPENED_DT'] - self.df['FID_DATE']).dt.days


        # Добавление AGE
        df_tmp = pd.merge(self.df[['SERIAL_NUM', 'FID', 'OPENED_DT']],
                          self.db_names,
                          how='left',
                          on=['FID'],
                          suffixes= ('', '_NAME'))

        df_tmp = df_tmp.loc[df_tmp['OPENED_DT'] >= df_tmp['FILE_SINCE_DT']]
        df_tmp.sort_values(by=['SERIAL_NUM', 'FILE_SINCE_DT'], ascending=[True, False], inplace=True)
        df_tmp.drop_duplicates(subset=['SERIAL_NUM'], keep='first', inplace=True)

        df_tmp['AGE'] = round((df_tmp['OPENED_DT'] - df_tmp['BIRTH_DT']).dt.days/365,0).astype('int')

        df_tmp.set_index('SERIAL_NUM', inplace=True)
        self.df['AGE'] = self.df['SERIAL_NUM'].map(df_tmp['AGE'].to_dict())

        # Добавление GENDER
        df_tmp = pd.merge(self.df[['SERIAL_NUM', 'FID', 'OPENED_DT']],
                          self.db_names,
                          how='left',
                          on=['FID'],
                          suffixes= ('', '_NAME'))

        df_tmp = df_tmp[~df_tmp['GENDER'].isnull()]
        df_tmp = df_tmp[df_tmp['OPENED_DT'] >= df_tmp['FILE_SINCE_DT']]
        df_tmp.sort_values(by=['SERIAL_NUM', 'FILE_SINCE_DT'], ascending=[True, False], inplace=True)
        df_tmp.drop_duplicates(subset=['SERIAL_NUM'], keep='first', inplace=True)
        df_tmp.set_index('SERIAL_NUM', inplace=True)

        self.df['GENDER'] = self.df['SERIAL_NUM'].map(df_tmp['GENDER'].to_dict())
        
        del(df_tmp)
    
    
    # Функция добавление агрегатов FID_DATE и FID_DAYS  
    def agg_db_info(self):
        
        self.print_log('INFORMATION Сбор агрегатов', level=1, timestamp=True)
        df_tmp = pd.merge(self.df[['SERIAL_NUM', 'FID', 'OPENED_DT']],
                          self.db_info,
                          how='inner',
                          on=['FID'],
                          suffixes= ('', '_INFO'))

        df_tmp = df_tmp[df_tmp['OPENED_DT'] > df_tmp['APP_DATE']]

        df_tmp.sort_values(by = ['FID', 'SERIAL_NUM', 'APP_DATE'], inplace=True)

        df_tmp['DAYS_GAP'] = (df_tmp['OPENED_DT'] - df_tmp['APP_DATE']).dt.days.astype('int')

        df_tmp['REQUESTS_1M_CNT'] = (df_tmp['DAYS_GAP'] <= 30)
        df_tmp['REQUESTS_3M_CNT'] = (df_tmp['DAYS_GAP'] <= 90)
        df_tmp['REQUESTS_6M_CNT'] = (df_tmp['DAYS_GAP'] <= 180)
        df_tmp['REQUESTS_12M_CNT'] = (df_tmp['DAYS_GAP'] <= 360)
        df_tmp['REQUESTS_24M_CNT'] = (df_tmp['DAYS_GAP'] <= 720)
        df_tmp['REQUESTS_36M_CNT'] = (df_tmp['DAYS_GAP'] <= 1080)

        LOAN_TYPE_LIST =  [101, 102, 201, 202, 203, 204, 301, 302, 303, 304, 305, 306, 401, 402, 403, 404, 405, 406, 407, 408, 451, 501, 601, 999]
        for LOAN_TYPE in LOAN_TYPE_LIST:
            df_tmp['REQUESTS_{}_1M_CNT'.format(LOAN_TYPE)] = ((df_tmp['DAYS_GAP'] <= 30) & (df_tmp['LOAN_TYPE']==LOAN_TYPE)).astype('bool')
            df_tmp['REQUESTS_{}_3M_CNT'.format(LOAN_TYPE)] = ((df_tmp['DAYS_GAP'] <= 90) & (df_tmp['LOAN_TYPE']==LOAN_TYPE)).astype('bool')

        df_tmp['REQUESTS_CT1_1M_CNT'] = ((df_tmp['DAYS_GAP'] <= 30) & (df_tmp['CREDITOR_TYPE']==1)).astype('bool')
        df_tmp['REQUESTS_CT2_1M_CNT'] = ((df_tmp['DAYS_GAP'] <= 30) & (df_tmp['CREDITOR_TYPE']==2)).astype('bool')
        df_tmp['REQUESTS_CT3_1M_CNT'] = ((df_tmp['DAYS_GAP'] <= 30) & (df_tmp['CREDITOR_TYPE']==3)).astype('bool')
        df_tmp['REQUESTS_CT4_1M_CNT'] = ((df_tmp['DAYS_GAP'] <= 30) & (df_tmp['CREDITOR_TYPE']==4)).astype('bool')
        df_tmp['REQUESTS_CT1_3M_CNT'] = ((df_tmp['DAYS_GAP'] <= 90) & (df_tmp['CREDITOR_TYPE']==1)).astype('bool')
        df_tmp['REQUESTS_CT2_3M_CNT'] = ((df_tmp['DAYS_GAP'] <= 90) & (df_tmp['CREDITOR_TYPE']==2)).astype('bool')
        df_tmp['REQUESTS_CT3_3M_CNT'] = ((df_tmp['DAYS_GAP'] <= 90) & (df_tmp['CREDITOR_TYPE']==3)).astype('bool')
        df_tmp['REQUESTS_CT4_3M_CNT'] = ((df_tmp['DAYS_GAP'] <= 90) & (df_tmp['CREDITOR_TYPE']==4)).astype('bool')

        cols = df_tmp.loc[:,'REJECTED_AMT_RUB':].columns.drop('DAYS_GAP')

        # Сохранение проверочных даннык
        self.df_test_info = df_tmp[df_tmp['FID'].isin(self.df_test_FID)].copy()
        
        df_tmp = df_tmp.groupby(by='SERIAL_NUM')
        for col in cols:
            self.df[col] = self.df['SERIAL_NUM'].map(df_tmp.agg({col:np.sum})[col].to_dict()).fillna(0).astype('int') 
        

        
        del(df_tmp)
    
    
    # Сбор агрегатов из таблицы ACCOUNT
    def agg_db_account(self):
        
        self.print_log('ACCOUNT Сбор агрегатов', level=1, timestamp=True)
        
        # Слияние
        df_tmp = pd.merge(self.df[['SERIAL_NUM', 'FID', 'OPENED_DT', 'MEMBER_CODE']],
                          self.db_account,
                          how='inner',
                          on=['FID'],
                          suffixes= ('', '_ACC'))

        df_tmp = df_tmp[df_tmp['OPENED_DT'] > df_tmp['OPENED_DT_ACC']]
        df_tmp.sort_values(by = ['FID', 'SERIAL_NUM', 'OPENED_DT_ACC'], inplace=True)


        df_tmp['DAYS_GAP'] = (df_tmp['OPENED_DT'] - df_tmp['OPENED_DT_ACC']).dt.days.astype('int')

        df_tmp['REPEAT'] = (df_tmp['MEMBER_CODE'] == df_tmp['MEMBER_CODE_ACC'])
        df_tmp['REPEAT_CNT'] = (df_tmp['MEMBER_CODE'] == df_tmp['MEMBER_CODE_ACC'])
        df_tmp['MEMBER_CODE_CNT'] = df_tmp['MEMBER_CODE_ACC']    

        df_tmp['CREDIT_ALL_CNT'] = True
        df_tmp['CREDIT_CLOSED_CNT'] = (df_tmp['ACCT_RTE_DTE'] < df_tmp['OPENED_DT']) & (df_tmp['ACCT_RTE_CDE']!=0)
        df_tmp['CREDIT_CURR_OPEN_CNT'] = ~df_tmp['CREDIT_CLOSED_CNT'] 
        df_tmp['CREDIT_CLOSED_BEFORE_CNT'] = df_tmp['CREDIT_CLOSED_CNT'] & (df_tmp['ACCT_RTE_DTE'] <= df_tmp['PAYT_DUE_DTE']) & (df_tmp['ACCT_RTE_CDE'] == 13)

        df_tmp['CREDIT_ALL_SUM'] = df_tmp['CREDIT_LIMIT_RUB']
        df_tmp['CREDIT_SUM_AVG'] = df_tmp['CREDIT_LIMIT_RUB']
        df_tmp['CREDIT_SUM_AVG'][df_tmp['CREDIT_LIMIT_RUB'] == 0] = None

        df_tmp['CREDIT_CLOSED_SUM_AVG'] = None
        df_tmp['CREDIT_CLOSED_SUM_AVG'].loc[df_tmp['CREDIT_CLOSED_CNT'] & (df_tmp['CREDIT_LIMIT_RUB'] > 0)] = df_tmp['CREDIT_LIMIT_RUB']
        df_tmp['CREDIT_CLOSED_SUM_AVG'] = df_tmp['CREDIT_CLOSED_SUM_AVG'].astype('float')

        df_tmp['CREDIT_TYPE_CNT'] = df_tmp['ACCT_TYPE']    

        df_tmp['OPEN_CREDIT_1M_CNT'] = (df_tmp['DAYS_GAP'] <= 30) & (df_tmp['CREDIT_LIMIT_RUB'] > 0)
        df_tmp['OPEN_CREDIT_3M_CNT'] = (df_tmp['DAYS_GAP'] <= 90) & (df_tmp['CREDIT_LIMIT_RUB'] > 0) 
        df_tmp['OPEN_CREDIT_6M_CNT'] = (df_tmp['DAYS_GAP'] <= 180) & (df_tmp['CREDIT_LIMIT_RUB'] > 0)
        df_tmp['OPEN_CREDIT_12M_CNT'] = (df_tmp['DAYS_GAP'] <= 360) & (df_tmp['CREDIT_LIMIT_RUB'] > 0)
        df_tmp['OPEN_CREDIT_24M_CNT'] = (df_tmp['DAYS_GAP'] <= 720) & (df_tmp['CREDIT_LIMIT_RUB'] > 0)
        df_tmp['OPEN_CREDIT_36M_CNT'] = (df_tmp['DAYS_GAP'] <= 1080) & (df_tmp['CREDIT_LIMIT_RUB'] > 0)

        df_tmp['CLOSED_CREDIT_1M_CNT'] = ((df_tmp['OPENED_DT'] - df_tmp['ACCT_RTE_DTE']).dt.days <= 30) & df_tmp['CREDIT_CLOSED_CNT'] & (df_tmp['CREDIT_LIMIT_RUB'] > 0)
        df_tmp['CLOSED_CREDIT_3M_CNT'] = ((df_tmp['OPENED_DT'] - df_tmp['ACCT_RTE_DTE']).dt.days <= 90) & df_tmp['CREDIT_CLOSED_CNT'] & (df_tmp['CREDIT_LIMIT_RUB'] > 0)
        df_tmp['CLOSED_CREDIT_6M_CNT'] = ((df_tmp['OPENED_DT'] - df_tmp['ACCT_RTE_DTE']).dt.days <= 180) & df_tmp['CREDIT_CLOSED_CNT'] & (df_tmp['CREDIT_LIMIT_RUB'] > 0)
        df_tmp['CLOSED_CREDIT_12M_CNT'] = ((df_tmp['OPENED_DT'] - df_tmp['ACCT_RTE_DTE']).dt.days <= 360) & df_tmp['CREDIT_CLOSED_CNT'] & (df_tmp['CREDIT_LIMIT_RUB'] > 0)
        df_tmp['CLOSED_CREDIT_24M_CNT'] = ((df_tmp['OPENED_DT'] - df_tmp['ACCT_RTE_DTE']).dt.days <= 720) & df_tmp['CREDIT_CLOSED_CNT'] & (df_tmp['CREDIT_LIMIT_RUB'] > 0)
        df_tmp['CLOSED_CREDIT_36M_CNT'] = ((df_tmp['OPENED_DT'] - df_tmp['ACCT_RTE_DTE']).dt.days <= 1080) & df_tmp['CREDIT_CLOSED_CNT'] & (df_tmp['CREDIT_LIMIT_RUB'] > 0)    

        df_tmp['CREDIT_DAYS_AVG'] = None
        df_tmp['CREDIT_DAYS_AVG'].loc[df_tmp['CREDIT_CLOSED_CNT'] & (df_tmp['CREDIT_LIMIT_RUB'] > 0)] = (df_tmp['ACCT_RTE_DTE']-df_tmp['OPENED_DT_ACC']).dt.days
        df_tmp['CREDIT_DAYS_AVG'] = df_tmp['CREDIT_DAYS_AVG'].astype('float')
        df_tmp['CREDIT_DAYS_MAX'] = df_tmp['CREDIT_DAYS_AVG']
        df_tmp['CREDIT_DAYS_MIN'] = df_tmp['CREDIT_DAYS_AVG']

        df_tmp['LAST_OPEN_CREDIT_DAYS'] = None
        df_tmp['LAST_OPEN_CREDIT_DAYS'][df_tmp['CREDIT_LIMIT_RUB'] > 0] = df_tmp['DAYS_GAP']
        df_tmp['LAST_OPEN_CREDIT_DAYS'] = df_tmp['LAST_OPEN_CREDIT_DAYS'].astype('float')
        df_tmp['FIRST_OPEN_CREDIT_DAYS'] = df_tmp['LAST_OPEN_CREDIT_DAYS']

        df_tmp['LAST_CARD_USAGE_PERIOD_DAYS'] = None
        df_tmp['LAST_CARD_USAGE_PERIOD_DAYS'].loc[(df_tmp['ACCT_TYPE'] == 7) & (df_tmp['CREDIT_LIMIT_RUB'] > 0)] = df_tmp['DAYS_GAP'] 
        df_tmp['LAST_CARD_USAGE_PERIOD_DAYS'] = df_tmp['LAST_CARD_USAGE_PERIOD_DAYS'].astype('float')

        df_tmp['MICRO_CREDIT_ALL_CNT'] = (df_tmp['ACCT_TYPE'] == 16) & (df_tmp['CREDIT_LIMIT_RUB'] > 0)
        df_tmp['MICRO_CREDIT_12M_CNT'] = (df_tmp['DAYS_GAP'] <= 360) & (df_tmp['ACCT_TYPE'] == 16) & (df_tmp['CREDIT_LIMIT_RUB'] > 0)
        df_tmp['MICRO_CREDIT_24M_CNT'] = (df_tmp['DAYS_GAP'] <= 720) & (df_tmp['ACCT_TYPE'] == 16) & (df_tmp['CREDIT_LIMIT_RUB'] > 0)
        df_tmp['MICRO_CREDIT_36M_CNT'] = (df_tmp['DAYS_GAP'] <= 1080) & (df_tmp['ACCT_TYPE'] == 16) & (df_tmp['CREDIT_LIMIT_RUB'] > 0)

        df_tmp['PAYMTPOS'] = (df_tmp['PAYMT_PAT_START_DT'].dt.year - df_tmp['OPENED_DT'].dt.year)*12 + (df_tmp['PAYMT_PAT_START_DT'].dt.month - df_tmp['OPENED_DT'].dt.month)
        df_tmp['PAYMTPOS_LAST_12M'] = 12
        df_tmp['PAYMTPOS_LAST_12M'][df_tmp['PAYMTPOS'] < 0] = df_tmp['PAYMTPOS_LAST_12M'] + df_tmp['PAYMTPOS']
        df_tmp['PAYMTPOS_LAST_12M'][df_tmp['PAYMTPOS_LAST_12M'] < 0] = 0
        df_tmp['PAYMTPOS'][df_tmp['PAYMTPOS'] < 0] = 0

        df_tmp['zip_pat'] = list(zip(df_tmp['PAYMT_PAT'], df_tmp['PAYMTPOS']))

        self.print_log('Расчет PAYMT_PAT_RETRO', level=2, timestamp=True)         
        df_tmp['PAYMT_PAT_RETRO'] = df_tmp['zip_pat'].map(lambda zip_pat: zip_pat[0][zip_pat[1]:])

        df_tmp.drop(columns='zip_pat', inplace=True)

        df_tmp['zip_pat_12m'] = list(zip(df_tmp['PAYMT_PAT_RETRO'], df_tmp['PAYMTPOS_LAST_12M']))

        self.print_log('Расчет PAYMT_PAT_RETRO_LAST_12M', level=2, timestamp=True)         
        df_tmp['PAYMT_PAT_RETRO_LAST_12M'] = df_tmp['zip_pat_12m'].map(lambda zip_pat: zip_pat[0][:zip_pat[1]])

        df_tmp.drop(columns='zip_pat_12m', inplace=True)

        df_tmp['BAD_CREDIT_ALL_CNT'] = (df_tmp['ACCT_RTE_DTE'] < df_tmp['OPENED_DT']) & (df_tmp['ACCT_RTE_CDE'].isin([21, 61, 90, 95, 96])) & (df_tmp['CREDIT_LIMIT_RUB'] > 500)
        df_tmp['BAD_CREDIT_12M_CNT'] = ((df_tmp['OPENED_DT'] - df_tmp['ACCT_RTE_DTE']).dt.days <360) & (df_tmp['ACCT_RTE_DTE'] < df_tmp['OPENED_DT']) & (df_tmp['ACCT_RTE_CDE'].isin([21, 61, 90, 95, 96])) & (df_tmp['CREDIT_LIMIT_RUB'] > 500)
        df_tmp['BAD_CREDIT_24M_CNT'] = ((df_tmp['OPENED_DT'] - df_tmp['ACCT_RTE_DTE']).dt.days <720) & (df_tmp['ACCT_RTE_DTE'] < df_tmp['OPENED_DT']) & (df_tmp['ACCT_RTE_CDE'].isin([21, 61, 90, 95, 96])) & (df_tmp['CREDIT_LIMIT_RUB'] > 500)
        df_tmp['BAD_CREDIT_36M_CNT'] = ((df_tmp['OPENED_DT'] - df_tmp['ACCT_RTE_DTE']).dt.days <1080) & (df_tmp['ACCT_RTE_DTE'] < df_tmp['OPENED_DT']) & (df_tmp['ACCT_RTE_CDE'].isin([21, 61, 90, 95, 96])) & (df_tmp['CREDIT_LIMIT_RUB'] > 500)

        self.print_log('Расчет PMT_ALL_CNT', level=2, timestamp=True)        
        df_tmp['PMT_ALL_CNT'] = df_tmp['PAYMT_PAT_RETRO'].map(len)

        self.print_log('Расчет GOOD_PMT_ALL_CNT', level=2, timestamp=True)    
        df_tmp['GOOD_PMT_ALL_CNT'] = df_tmp['PAYMT_PAT_RETRO'].map(lambda pat: np.sum([pat.count(s) for s in '1']))

        self.print_log('Расчет BAD_PMT_ALL_CNT', level=2, timestamp=True)   
        df_tmp['BAD_PMT_ALL_CNT'] = df_tmp['PAYMT_PAT_RETRO'].map(lambda pat: np.sum([pat.count(s) for s in 'A2345789']))

        self.print_log('Расчет OVERDUE_CREDIT_ALL_CNT', level=2, timestamp=True)   
        df_tmp['OVERDUE_CREDIT_ALL_CNT'] = (df_tmp['PAYMT_PAT_RETRO'].map(lambda pat: np.sum([pat.count(s) for s in 'A2345789'])) > 0)

        self.print_log('Расчет OVERDUE_CREDIT_ALL_30_CNT', level=2, timestamp=True)   
        df_tmp['OVERDUE_CREDIT_ALL_30_CNT'] = (df_tmp['PAYMT_PAT_RETRO'].map(lambda pat: np.sum([pat.count(s) for s in '2345789'])) > 0)

        self.print_log('Расчет OVERDUE_CREDIT_ALL_60_CNT', level=2, timestamp=True)   
        df_tmp['OVERDUE_CREDIT_ALL_60_CNT'] = (df_tmp['PAYMT_PAT_RETRO'].map(lambda pat: np.sum([pat.count(s) for s in '345789'])) > 0)

        self.print_log('Расчет OVERDUE_CREDIT_ALL_90_CNT', level=2, timestamp=True)   
        df_tmp['OVERDUE_CREDIT_ALL_90_CNT'] = (df_tmp['PAYMT_PAT_RETRO'].map(lambda pat: np.sum([pat.count(s) for s in '45789'])) > 0)

        self.print_log('Расчет PMT_12M_CNT', level=2, timestamp=True)   
        df_tmp['PMT_12M_CNT'] = df_tmp['PAYMT_PAT_RETRO_LAST_12M'].map(len)

        self.print_log('Расчет GOOD_PMT_12M_CNT', level=2, timestamp=True)   
        df_tmp['GOOD_PMT_12M_CNT'] = df_tmp['PAYMT_PAT_RETRO_LAST_12M'].map(lambda pat: np.sum([pat.count(s) for s in '1']))

        self.print_log('Расчет BAD_PMT_12M_CNT', level=2, timestamp=True)   
        df_tmp['BAD_PMT_12M_CNT'] = df_tmp['PAYMT_PAT_RETRO_LAST_12M'].map(lambda pat: np.sum([pat.count(s) for s in 'A2345789']))

        self.print_log('Расчет OVERDUE_CREDIT_12M_CNT', level=2, timestamp=True)   
        df_tmp['OVERDUE_CREDIT_12M_CNT'] = (df_tmp['PAYMT_PAT_RETRO_LAST_12M'].map(lambda pat: np.sum([pat.count(s) for s in 'A2345789'])) > 0)

        self.print_log('Расчет OVERDUE_CREDIT_12M_30_CNT', level=2, timestamp=True)   
        df_tmp['OVERDUE_CREDIT_12M_30_CNT'] = (df_tmp['PAYMT_PAT_RETRO_LAST_12M'].map(lambda pat: np.sum([pat.count(s) for s in '2345789'])) > 0)

        self.print_log('Расчет OVERDUE_CREDIT_12M_60_CNT', level=2, timestamp=True)   
        df_tmp['OVERDUE_CREDIT_12M_60_CNT'] = (df_tmp['PAYMT_PAT_RETRO_LAST_12M'].map(lambda pat: np.sum([pat.count(s) for s in '345789'])) > 0)

        self.print_log('Расчет OVERDUE_CREDIT_12M_90_CNT', level=2, timestamp=True)   
        df_tmp['OVERDUE_CREDIT_12M_90_CNT'] = (df_tmp['PAYMT_PAT_RETRO_LAST_12M'].map(lambda pat: np.sum([pat.count(s) for s in '45789'])) > 0)

        self.print_log('Расчет OVERDUE_CREDIT_CURR_CNT', level=2, timestamp=True)
        df_tmp['OVERDUE_CREDIT_CURR_CNT'] = (df_tmp['PAYMT_PAT_RETRO'].map(lambda pat: np.sum([pat.count(s) for s in 'A2345789'])) > 0) & df_tmp['CREDIT_CURR_OPEN_CNT']

        df_tmp['OVERDUE_DURATION_DAYS_MAX'] = 0
        df_tmp['OVERDUE_DURATION_DAYS_MAX'].loc[(df_tmp['ACCT_RTE_DTE'] < df_tmp['OPENED_DT']) & (df_tmp['ACCT_RTE_CDE'] == 52) & (df_tmp['CREDIT_LIMIT_RUB'] > 500)] = (df_tmp['OPENED_DT'] - df_tmp['ACCT_RTE_DTE']).dt.days
        df_tmp['OVERDUE_DURATION_DAYS_MAX'] = df_tmp['OVERDUE_DURATION_DAYS_MAX'].astype('int')

        df_tmp['GOOD_CREDIT_CLOSED_CNT'] = (df_tmp['BAD_PMT_ALL_CNT'] == 0) & (df_tmp['CLOSED_DT'] < df_tmp['OPENED_DT'])    

        # Группировка и перенос данных
        cols = list(df_tmp.loc[:,'DAYS_GAP':].columns.drop(['DAYS_GAP', 'PAYMTPOS' ,'PAYMTPOS_LAST_12M', 'PAYMT_PAT_RETRO', 'PAYMT_PAT_RETRO_LAST_12M']))
        df_tmp_agg = df_tmp.groupby(by='SERIAL_NUM')

        for col in cols:
            if col in ['CREDIT_DAYS_MAX', 'FIRST_OPEN_CREDIT_DAYS', 'OVERDUE_DURATION_DAYS_MAX']:
                agg_func = np.max

            elif col in ['CREDIT_DAYS_MIN', 'LAST_OPEN_CREDIT_DAYS', 'LAST_CARD_USAGE_PERIOD_DAYS']:
                agg_func = np.min

            elif col in ['CREDIT_DAYS_AVG', 'CREDIT_SUM_AVG', 'CREDIT_CLOSED_SUM_AVG']:
                agg_func = np.mean

            elif col in ['REPEAT']:
                agg_func = lambda x: int(np.sum(x) > 0)

            elif col in ['CREDIT_TYPE_CNT', 'MEMBER_CODE_CNT']:
                agg_func = lambda x: len(np.unique(x))

            else:
                agg_func = np.sum
            
            self.print_log('Агрегация ' + col, level=2, timestamp=True)
            self.df[col] = self.df['SERIAL_NUM'].map(df_tmp_agg.agg({col:agg_func})[col].to_dict())
        
        # Сохранение проверочных даннык
        self.df_test_account = df_tmp[df_tmp['FID'].isin(self.df_test_FID)].copy()
        
        del(df_tmp)

    
    # Расчет флагов и дополнительных агрегатов
    def flag_calculation(self):
        
        self.print_log('Расчет флагов и дополнительных агрегатов', level=1, timestamp=True)
        self.df['CREDIT_PERIOD_DAYS'] = (self.df['PAYT_DUE_DTE'] - self.df['OPENED_DT']).dt.days

        self.df['CREDIT_WITH_DUE_RATE'] = self.df['OVERDUE_CREDIT_ALL_CNT'] / self.df['CREDIT_ALL_CNT'] * 100
        self.df['OVERDUE_CREDIT_CURR_RATE'] = self.df['OVERDUE_CREDIT_CURR_CNT'] / self.df['CREDIT_CURR_OPEN_CNT'] * 100

        self.df['BAD_PMT_ALL_RATE'] = self.df['BAD_PMT_ALL_CNT'] / self.df['PMT_ALL_CNT']
        self.df['BAD_PMT_12M_RATE'] = self.df['BAD_PMT_12M_CNT'] / self.df['PMT_12M_CNT']

        self.df['CREDIT_HISTORY'] = (~self.df['REPEAT'].isnull()).astype('int')

        #Заполнение нулей и преобразование типов
        for col in self.df.loc[:,'REPEAT':'BAD_PMT_12M_RATE'].columns:
            self.df[col].fillna(0, inplace=True)
            if str(self.df[col].dtype)[:5] == 'float' and np.sum(self.df[col]) == np.sum(self.df[col].astype(int)):
                self.df[col] = self.df[col].astype('int')
        
        self.df['REPEAT'] = self.df['REPEAT'].astype('int')
        self.df['CREDIT_DAYS_AVG'] = self.df['CREDIT_DAYS_AVG'].astype('int')
        self.df['CREDIT_SUM_AVG'] = self.df['CREDIT_SUM_AVG'].astype('int')
        self.df['CREDIT_ALL_SUM'] = self.df['CREDIT_ALL_SUM'].astype('int')        
        self.df['CREDIT_CLOSED_SUM_AVG'] = self.df['CREDIT_CLOSED_SUM_AVG'].astype('int')

        #Расчет Флагов
        self.print_log('Расчет 90_24_MOB', level=2, timestamp=True)  
        self.df['90_24_MOB']  = self.df['PAYMT_PAT'].map(lambda pat: np.sum([pat[-(24 if (pat[-1] =='0') else 23):][0].count(s) for s in '4589'])>0).astype('int')

        self.print_log('Расчет 90_24_EVER', level=2, timestamp=True)      
        self.df['90_24_EVER'] = self.df['PAYMT_PAT'].map(lambda pat: np.sum([pat[-(24 if (pat[-1] =='0') else 23):].count(s) for s in '4589'])>0).astype('int')

        self.print_log('Расчет 90_12_MOB', level=2, timestamp=True)    
        self.df['90_12_MOB']  = self.df['PAYMT_PAT'].map(lambda pat: np.sum([pat[-(12 if (pat[-1] =='0') else 11):][0].count(s) for s in '4589'])>0).astype('int')

        self.print_log('Расчет 0_12_EVER', level=2, timestamp=True)    
        self.df['90_12_EVER'] = self.df['PAYMT_PAT'].map(lambda pat: np.sum([pat[-(12 if (pat[-1] =='0') else 11):].count(s) for s in '4589'])>0).astype('int')

        self.print_log('Расчет 60_6_MOB', level=2, timestamp=True)       
        self.df['60_6_MOB']  = self.df['PAYMT_PAT'].map(lambda pat: np.sum([pat[-(6 if (pat[-1] =='0') else 5):][0].count(s) for s in '34589'])>0).astype('int')

        self.print_log('Расчет 60_6_EVER', level=2, timestamp=True)           
        self.df['60_6_EVER'] = self.df['PAYMT_PAT'].map(lambda pat: np.sum([pat[-(6 if (pat[-1] =='0') else 5):].count(s) for s in '34589'])>0).astype('int')

        self.print_log('Расчет 60_12_MOB', level=2, timestamp=True)    
        self.df['60_12_MOB']  = self.df['PAYMT_PAT'].map(lambda pat: np.sum([pat[-(12 if (pat[-1] =='0') else 11):][0].count(s) for s in '34589'])>0).astype('int')

        self.print_log('Расчет 60_12_EVER', level=2, timestamp=True)    
        self.df['60_12_EVER'] = self.df['PAYMT_PAT'].map(lambda pat: np.sum([pat[-(12 if (pat[-1] =='0') else 11):].count(s) for s in '34589'])>0).astype('int')

        self.print_log('Расчет 30_3_MOB', level=2, timestamp=True) 
        self.df['30_3_MOB']  = self.df['PAYMT_PAT'].map(lambda pat: np.sum([pat[-(3 if (pat[-1] =='0') else 2):][0].count(s) for s in '234589'])>0).astype('int')

        self.print_log('Расчет 30_3_EVER', level=2, timestamp=True)      
        self.df['30_3_EVER'] = self.df['PAYMT_PAT'].map(lambda pat: np.sum([pat[-(3 if (pat[-1] =='0') else 2):].count(s) for s in '234589'])>0).astype('int')

        self.print_log('Расчет FPD', level=2, timestamp=True) 
        self.df['FPD'] = self.df['PAYMT_PAT'].map(lambda pat: np.sum([pat.rstrip('0')[-1:].count(s) for s in 'A2345789']) > 0).astype('int')

        self.print_log('Расчет SPD', level=2, timestamp=True) 
        self.df['SPD'] = self.df['PAYMT_PAT'].map(lambda pat: np.sum([pat.rstrip('0')[-2:].count(s) for s in 'A2345789']) > 0).astype('int')

        self.print_log('Расчет TPD', level=2, timestamp=True)   
        self.df['TPD'] = self.df['PAYMT_PAT'].map(lambda pat: np.sum([pat.rstrip('0')[-3:].count(s) for s in 'A2345789']) > 0).astype('int')

        # Корректировка флагов в случае если флаг дефолта не вызрел (длинна PAYMT_PAT не меньше периода флага и кредит еще открыт)
        self.df['90_24_MOB'][self.df['PAYMT_PAT'].map(lambda pat: len(pat) < (24 if (pat[-1] =='0') else 23)) & (self.df['ACCT_RTE_CDE'] == 0)] = None
        self.df['90_24_EVER'][self.df['PAYMT_PAT'].map(lambda pat: len(pat) < (24 if (pat[-1] =='0') else 23)) & (self.df['ACCT_RTE_CDE'] == 0)] = None

        self.df['90_12_MOB'][self.df['PAYMT_PAT'].map(lambda pat: len(pat) < (12 if (pat[-1] =='0') else 11)) & (self.df['ACCT_RTE_CDE'] == 0)] = None
        self.df['90_12_EVER'][self.df['PAYMT_PAT'].map(lambda pat: len(pat) < (12 if (pat[-1] =='0') else 11)) & (self.df['ACCT_RTE_CDE'] == 0)] = None

        self.df['60_12_MOB'][self.df['PAYMT_PAT'].map(lambda pat: len(pat) < (12 if (pat[-1] =='0') else 11)) & (self.df['ACCT_RTE_CDE'] == 0)] = None
        self.df['60_12_EVER'][self.df['PAYMT_PAT'].map(lambda pat: len(pat) < (12 if (pat[-1] =='0') else 11)) & (self.df['ACCT_RTE_CDE'] == 0)] = None   

        self.df['60_6_MOB'][self.df['PAYMT_PAT'].map(lambda pat: len(pat) < (6 if (pat[-1] =='0') else 5)) & (self.df['ACCT_RTE_CDE'] == 0)] = None
        self.df['60_6_EVER'][self.df['PAYMT_PAT'].map(lambda pat: len(pat) < (6 if (pat[-1] =='0') else 5)) & (self.df['ACCT_RTE_CDE'] == 0)] = None

        self.df['30_3_MOB'][self.df['PAYMT_PAT'].map(lambda pat: len(pat) < (4 if (pat[-1] =='0') else 3)) & (self.df['ACCT_RTE_CDE'] == 0)] = None
        self.df['30_3_EVER'][self.df['PAYMT_PAT'].map(lambda pat: len(pat) < (4 if (pat[-1] =='0') else 3)) & (self.df['ACCT_RTE_CDE'] == 0)] = None

        self.df['FPD'][self.df['PAYMT_PAT'].map(lambda pat: len(pat.rstrip('0')[-1:]) < 1)] = None
        self.df['SPD'][self.df['PAYMT_PAT'].map(lambda pat: len(pat.rstrip('0')[-2:]) < 2) & (self.df['ACCT_RTE_CDE'] == 0)] = None
        self.df['TPD'][self.df['PAYMT_PAT'].map(lambda pat: len(pat.rstrip('0')[-3:]) < 3) & (self.df['ACCT_RTE_CDE'] == 0)] = None

# **СБОР АГРЕГАТОВ**

### BANK_90 2017.12-2019.02

In [3]:
period_dict = {'201902': ['2019-02-01', '2019-02-28'],
               '201901': ['2019-01-01', '2019-01-31'],
               '201812': ['2018-12-01', '2018-12-31'],
               '201811': ['2018-11-01', '2018-11-30'],
               '201810': ['2018-10-01', '2018-10-31'],
               '201809': ['2018-09-01', '2018-09-30'],
               '201808': ['2018-08-01', '2018-08-31'],
               '201807': ['2018-07-01', '2018-07-31'],
               '201806': ['2018-06-01', '2018-06-30'],
               '201805': ['2018-05-01', '2018-05-31'],
               '201804': ['2018-04-01', '2018-04-30'],
               '201803': ['2018-03-01', '2018-03-31'],
               '201802': ['2018-02-01', '2018-02-28'],               
               '201801': ['2018-01-01', '2018-01-31'], 
               '201712': ['2017-12-01', '2017-12-31']}

PATH_TO_DATA = 'Z:/ushakov/(PROJECTS)/bank_90/(data)/FID_201712-201902/'

account_csv_file_name = 'FID_201712-201902_ACCOUNT.csv'
information_csv_file_name = 'FID_201712-201902_INFORMATION.csv'
name_csv_file_name = 'FID_201712-201902_NAME.csv'
address_csv_file_name = 'FID_201712-201902_ADDRESS.csv'
cbrates_csv_file_name = 'CBRATES.csv'

bank_90_agg_collector = AggCollector(acct_type_list=[1, 6, 7, 9],
                                     period_dict=period_dict,
                                     PATH_TO_DATA=PATH_TO_DATA,
                                     account_csv_file_name=account_csv_file_name, 
                                     information_csv_file_name=information_csv_file_name, 
                                     name_csv_file_name=name_csv_file_name, 
                                     address_csv_file_name=address_csv_file_name, 
                                     cbrates_csv_file_name=cbrates_csv_file_name)

In [ ]:
# Сбор агрегатов по всем займам
bank_90_agg_collector.collect(check_data_export=False,
                              collect_rejects=False)

In [ ]:
# Сбор агрегатов по отказным займам
bank_90_agg_collector.collect(periods_to_collect_list=['201902', '201901', '201812'],
                              check_data_export=False,
                              collect_rejects=True)

РАСЧЕТ АГРЕГАТОВ

ACCOUNT Загрузка ключей таблицы   ....................................   2020-03-16 16:32:23
INFORMATION Загрузка ключей таблицы   ................................   2020-03-16 16:40:18
ADDRESS Загрузка таблицы   ...........................................   2020-03-16 16:47:50
ADDRESS Предобработка данных   .......................................   2020-03-16 17:03:32
NAME Загрузка таблицы   ..............................................   2020-03-16 17:26:11
NAME Предобработка таблицы   .........................................   2020-03-16 17:27:19
CBRATES Загрузка таблицы   ...........................................   2020-03-16 17:29:04
CBRATES Предобработка таблицы   ......................................   2020-03-16 17:29:04

ПЕРИОД 2019-02-01 - 2019-02-28:
    ACCOUNT Сбор FID (2019-02-01 - 2019-02-28)   .....................   2020-03-16 17:29:20
    ACCOUNT Выборка ключей для загрузки   ............................   2020-03-16 17:30:40
    ACCOUNT Загрузк

In [ ]:
bank_90_agg_collector.get_sample_data()

### BANK_90 SAMPLE 2017.12-2019.02

In [ ]:
period_dict = {'201902': ['2019-02-01', '2019-02-28'],
               '201901': ['2019-01-01', '2019-01-31'],
               '201812': ['2018-12-01', '2018-12-31'],
               '201811': ['2018-11-01', '2018-11-30'],
               '201810': ['2018-10-01', '2018-10-31'],
               '201809': ['2018-09-01', '2018-09-30'],
               '201808': ['2018-08-01', '2018-08-31'],
               '201807': ['2018-07-01', '2018-07-31'],
               '201806': ['2018-06-01', '2018-06-30'],
               '201805': ['2018-05-01', '2018-05-31'],
               '201804': ['2018-04-01', '2018-04-30'],
               '201803': ['2018-03-01', '2018-03-31'],
               '201802': ['2018-02-01', '2018-02-28'],               
               '201801': ['2018-01-01', '2018-01-31'], 
               '201712': ['2017-12-01', '2017-12-31']}

PATH_TO_DATA = 'Z:/ushakov/(PROJECTS)/bank_90/(data)/FID_201712-201902/SAMPLE/'

account_csv_file_name = 'FID_201712-201902_ACCOUNT_SAMPLE.csv'
information_csv_file_name = 'FID_201712-201902_INFORMATION_SAMPLE.csv'
name_csv_file_name = 'FID_201712-201902_NAME_SAMPLE.csv'
address_csv_file_name = 'FID_201712-201902_ADDRESS_SAMPLE.csv'
cbrates_csv_file_name = 'CBRATES_SAMPLE.csv'

bank_90_sample_agg_collector = AggCollector(acct_type_list=[1, 6, 7, 9],
                                             period_dict=period_dict,
                                             PATH_TO_DATA=PATH_TO_DATA,
                                             account_csv_file_name=account_csv_file_name, 
                                             information_csv_file_name=information_csv_file_name, 
                                             name_csv_file_name=name_csv_file_name, 
                                             address_csv_file_name=address_csv_file_name, 
                                             cbrates_csv_file_name=cbrates_csv_file_name)

In [ ]:
# Сбор агрегатов по всем займам
bank_90_sample_agg_collector.collect(check_data_export=False,
                                     collect_rejects=False)

In [ ]:
# Сбор агрегатов по отказным займам
bank_90_sample_agg_collector.collect(periods_to_collect_list=['201902', '201901', '201812'],
                                     check_data_export=False,
                                     collect_rejects=True)

### MFO 2019.07.01-2019.10.31

In [ ]:
period_dict = {'20190701': ['2019-07-01', '2019-07-15'],
               '20190716': ['2019-07-16', '2019-07-31'],
               '20190801': ['2019-08-01', '2019-08-15'],
               '20190816': ['2019-08-16', '2019-08-31'],
               '20190901': ['2019-09-01', '2019-09-15'],
               '20190916': ['2019-09-16', '2019-09-30'],
               '20191001': ['2019-10-01', '2019-10-15'],
               '20191016': ['2019-10-16', '2019-10-31']}

PATH_TO_DATA = 'Z:/ushakov/(PROJECTS)/MFO/(data)/FID_20190701-20191031/'

account_csv_file_name = 'FID_20190701-20191031_ACCOUNT.csv'
information_csv_file_name = 'FID_20190701-20191031_INFORMATION.csv'
name_csv_file_name = 'FID_20190701-20191031_NAME.csv'
address_csv_file_name = 'FID_20190701-20191031_ADDRESS.csv'
cbrates_csv_file_name = 'CBRATES.csv'

MFO_agg_collector = AggCollector(acct_type_list=[16],
                                 period_dict=period_dict,
                                 PATH_TO_DATA=PATH_TO_DATA,
                                 account_csv_file_name=account_csv_file_name, 
                                 information_csv_file_name=information_csv_file_name, 
                                 name_csv_file_name=name_csv_file_name, 
                                 address_csv_file_name=address_csv_file_name, 
                                 cbrates_csv_file_name=cbrates_csv_file_name)

In [ ]:
MFO_agg_collector.collect()